In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
comment_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/new_comment_list_20220122.csv')
comment_df.head()

,Unnamed: 0,comment,label
0,0,이재명 좋으면 엄지 척\n윤석열 좋으면 누르던지 말던지..^^,1
1,1,조국 집안 비리나 범죄와 윤석열이 집안 비리나 범죄 행위중 어느 집안이 더 중 범죄...,1
2,2,드디어 TK보수도 이재명을 제대로 알아보는 사람들이 나타나기 시작했다.\n현명한 국...,1
3,3,드디어 언론이 시작했군.. 참 기레기들 김건희나 이렇게 조사좀 하시지... 쥴리 시...,1
4,4,이재명이 될까 두려움보다도 윤석렬이 되면 이 나라가 어떡하지의 걱정이 더 크다~!,1


In [ ]:
comment_df = comment_df.sample(frac=1)  # row 전체 shuffle

comment_df = comment_df[['comment', 'label']]
comment_df.index = range(len(comment_df))
comment_df.head()

,comment,label
0,국짐은 비대위원장에\n진 중권이나 서 민을 데려다\n써라!!\n딱 어울린다!!,1
1,방탄을 독도수비대로 보내라ᆢ그러면 자연스럽게 독도는 우리땅이라는것을 전세계인이 알게...,0
2,36계 줄행랑 ㅋㅋ,0
3,5공이 살기 좋았다? 필리핀보다 가난했고 집에 포니 한대 없었다 남이랑 부대 끼며 ...,0
4,진짜 bts는 면제해줘야되는 거 아니냐. 운동선수들 메달에만 급급해서 졸전해도 메달...,1


train / test set으로 분할 및 train을 wordvec로 변경

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(comment_df, test_size=0.2)

In [ ]:
len(comment_df), len(train), len(test), round(len(train)/len(comment_df) , 2) , round(len(test)/len(comment_df) , 2)

(262682, 210145, 52537, 0.8, 0.2)

In [ ]:
train_token = []
for i, row in tqdm(train.iterrows(), total=len(train)):
    comment = row['comment']
    token_list = comment.split()
    syllable_list = []
    for token in token_list:
        syllable_list.extend([t for t in token])
    train_token.append(syllable_list)
train_token[0]

In [ ]:
model = Word2Vec(sentences=train_token, size=300, window=5, min_count=1, workers=4)
model.save("/content/drive/My Drive/Colab Notebooks/data/word2vec_comment_20220122.model")

In [ ]:
model = Word2Vec.load("/content/drive/My Drive/Colab Notebooks/data/word2vec_comment_20220122.model")

refine_token(train) => word vector avg 로 표현

In [ ]:
# comment -> vector (syllable)
def comment2vec(model = None, comment_list = None):
    if model == None:
        return []
    else:
        if comment_list == None:
            return []
        else:
            unk_vec = np.zeros(model.trainables.layer1_size, dtype="float32")
            comment_vec_list = []
            for comment in tqdm(comment_list, total=len(comment_list)):
                token_list = comment.split()
                comment_vec = []
                for token in token_list:
                    syllable_list = [t for t in token]
                    for syllable in syllable_list:
                        vec = []
                        if syllable in model.wv:
                            vec = model.wv[syllable]
                        else:
                            vec = unk_vec
                        comment_vec.append(vec)
                comment_vec = np.average(comment_vec, axis=0)
                comment_vec_list.append(comment_vec)

            return comment_vec_list

In [ ]:
train_vec = comment2vec(model= model, comment_list = [comment for comment in train['comment']])
test_vec = comment2vec(model = model, comment_list = [comment for comment in test['comment']])

len(train_vec)

  0%|          | 0/210145 [00:00<?, ?it/s]

  0%|          | 0/52537 [00:00<?, ?it/s]

210145

In [ ]:
# train_vec_list = np.array(train_vec_list, dtype='object')
# test_vec_list = np.array(test_vec_list, dtype='object')

In [ ]:
train_label = [ [label] for label in train['label']]
test_label = [ [label] for label in test['label']]

In [ ]:
len(train_label), len(test_label)

(210145, 52537)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
train_vec_list = np.array(train_vec)
x_train = torch.FloatTensor(train_vec_list)

train_label = np.array(train_label)
y_train = torch.FloatTensor(train_label)

test_vec_list = np.array(test_vec)
x_test = torch.FloatTensor(test_vec_list)

test_label = np.array(test_label)
y_test = torch.FloatTensor(test_label)

In [ ]:
print(x_train.shape)
print(y_train.shape)

torch.Size([210145, 300])
torch.Size([210145, 1])


In [ ]:
# 모델 초기화
W = torch.zeros((model.trainables.layer1_size, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost =  - ((y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean())

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693147
Epoch 1000/10000 Cost: 0.665021
Epoch 2000/10000 Cost: 0.664365
Epoch 3000/10000 Cost: 0.664047
Epoch 4000/10000 Cost: 0.663853
Epoch 5000/10000 Cost: 0.663718
Epoch 6000/10000 Cost: 0.663619
Epoch 7000/10000 Cost: 0.663542
Epoch 8000/10000 Cost: 0.663479
Epoch 9000/10000 Cost: 0.663428
Epoch 10000/10000 Cost: 0.663385


In [ ]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction

tensor([[ True],
        [ True],
        [False],
        ...,
        [ True],
        [ True],
        [ True]])

In [ ]:
true_count = 0
false_count = 0
for label, pred in zip(train_label, prediction):
    if (label == 0 and pred.item() == False) or (label == 1 and pred.item() == True):
        true_count += 1
    else:
        false_count += 1
true_count, false_count

(126603, 83542)

In [ ]:
round((true_count / len(train_label) * 100) , 2)

60.25

In [ ]:
hypothesis = torch.sigmoid(x_test.matmul(W) + b)
prediction = hypothesis >= torch.FloatTensor([0.5])

true_count = 0
false_count = 0
for label, pred in zip(test_label, prediction):
    if (label == 0 and pred.item() == False) or (label == 1 and pred.item() == True):
        true_count += 1
    else:
        false_count += 1
true_count, false_count

(31742, 20795)

In [ ]:
round((true_count / len(test_label) * 100) , 2)

60.42

In [ ]:
def commentClassification(txt):
    txt_vec = comment2vec(model, [txt])
    input = torch.FloatTensor(txt_vec[0])

    hypothesis = torch.sigmoid(input.matmul(W) + b)
    prediction = hypothesis >= torch.FloatTensor([0.5])
    if prediction.item() == False:
        print("daum comment")
    else:
        print("naver comment")

In [ ]:
txt = "문재인 대통령님 최고!"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


In [ ]:
txt = "문재앙 뒤져라"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


In [ ]:
txt = "이재명 당선"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

naver comment


In [ ]:
txt = "찢재명 조폭"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


nn.module 실습

In [ ]:
torch.manual_seed(1)

torch_model = nn.Sequential(
    nn.Linear(model.trainables.layer1_size, 1),
    nn.Sigmoid()
)

In [ ]:
torch_model(x_train)

tensor([[0.5385],
        [0.4898],
        [0.4851],
        ...,
        [0.5196],
        [0.4904],
        [0.5375]], grad_fn=<SigmoidBackward0>)

In [ ]:
optimizer = optim.SGD(torch_model.parameters(), lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = torch_model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 1000 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/10000 Cost: 0.664989 Accuracy 60.00%
Epoch 1000/10000 Cost: 0.664354 Accuracy 60.13%
Epoch 2000/10000 Cost: 0.664042 Accuracy 60.16%
Epoch 3000/10000 Cost: 0.663849 Accuracy 60.19%
Epoch 4000/10000 Cost: 0.663715 Accuracy 60.22%
Epoch 5000/10000 Cost: 0.663616 Accuracy 60.24%
Epoch 6000/10000 Cost: 0.663539 Accuracy 60.25%
Epoch 7000/10000 Cost: 0.663477 Accuracy 60.24%
Epoch 8000/10000 Cost: 0.663425 Accuracy 60.25%
Epoch 9000/10000 Cost: 0.663382 Accuracy 60.26%
Epoch 10000/10000 Cost: 0.663345 Accuracy 60.25%


In [ ]:
# test 
hypothesis = torch_model(x_test)
prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
correct_prediction = prediction.float() == y_test # 실제값과 일치하는 경우만 True로 간주
accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
print('Accuracy {:2.2f}%'.format(accuracy * 100))

Accuracy 60.45%


In [ ]:
def commentClassification(txt):
    txt_vec = comment2vec(model, [txt])
    input = torch.FloatTensor(txt_vec[0])

    hypothesis = torch_model(input)
    prediction = hypothesis >= torch.FloatTensor([0.5])
    if prediction.item() == False:
        print("daum comment")
    else:
        print("naver comment")

In [ ]:
txt = "문재인 대통령님!"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


In [ ]:
txt = "문재앙 새끼"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


In [ ]:
txt = "이재명 의원님"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

naver comment


In [ ]:
txt = "찢재명 새끼"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

daum comment


In [ ]:
txt = "윤석열 의원님"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

naver comment


In [ ]:
txt = "윤꼴통 새끼"
commentClassification(txt)

  0%|          | 0/1 [00:00<?, ?it/s]

naver comment
